
# **数据挖掘——Home Credit Default Risk**

Authors：李林（3120220938）、杨洋（3220211141）、敬甲男（3220221052）、李翰杰（3120220936）

github地址：https://github.com/leealim/kaggle-Home-Credit-Default-Risk

---

## 数据预处理——数据分析

共八张表，逐个进行分析：
- application_{train|test}.csv:客户申请表
- bureau.csv/bureau_balance.csv:客户历史借款记录
- POS_CASH_balance.csv:客户POS和现金贷款历史
- credit_card_balance.csv:客户信用卡的snapshot历史
- previous_application.csv:客户历史申请记录
- installments_payments.csv:客户信用卡还款记录

---


In [177]:
#引入本部分所需要的包，并定义需要的值和函数

import pandas as pd
import matplotlib.pyplot as plt

app_tr_path="..\\data\\home-credit-default-risk\\application_train.csv"
app_te_path="..\\data\\home-credit-default-risk\\application_test.csv"
bur_path="..\\data\\home-credit-default-risk\\bureau.csv"
bur_bal_path="..\\data\\home-credit-default-risk\\bureau_balance.csv"
pos_path="..\\data\\home-credit-default-risk\\POS_CASH_balance.csv"
cre_path="..\\data\\home-credit-default-risk\\credit_card_balance.csv"
pre_path="..\\data\\home-credit-default-risk\\previous_application.csv"
ins_path="..\\data\\home-credit-default-risk\\installments_payments.csv"

hom_path="..\\data\\home-credit-default-risk\\HomeCredit_columns_description.csv"# 列描述表
hom=pd.read_csv(hom_path)
app_tr=pd.read_csv(app_tr_path)

def missing_values_table(df,table_name):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table = mis_val_table.rename(
        columns = {0 : 'Missing Values',
        1 : '% of Total Values'})
    mis_val_table = mis_val_table.sort_values(
    '% of Total Values', ascending=False).round(1)
    miss_num=(mis_val_table["Missing Values"]!=0).sum()
    print("Total "+ str(miss_num) +" columns missing values")
    mis_val_table=mis_val_table.drop(index=mis_val_table[miss_num:].index)

    mis_val_table=mis_val_table.merge(hom,how="left",left_index=True,right_on='Row')
    mis_val_table=mis_val_table.drop(columns=['Unnamed: 0'])
    mis_val_table=mis_val_table.drop(index=mis_val_table.loc[mis_val_table["Table"]!=table_name].index)
    mis_val_table=mis_val_table.reindex(columns=["Row","Description","Special","Missing Values","% of Total Values"])
    mis_val_table = mis_val_table.reset_index(drop=True)

    return mis_val_table



### 1. **application_{train|test}.csv**

In [178]:
# 查看训练数据的基本数据特征

app_tr=pd.read_csv(app_tr_path)
app_tr.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [179]:
# 缺失值分析

t=missing_values_table(app_tr,"application_{train|test}.csv")
t


Total 67 columns missing values


,Row,Description,Special,Missing Values,% of Total Values
0,COMMONAREA_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9
1,COMMONAREA_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9
2,COMMONAREA_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9
3,NONLIVINGAPARTMENTS_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,213514,69.4
4,NONLIVINGAPARTMENTS_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,213514,69.4
...,...,...,...,...,...
62,EXT_SOURCE_2,Normalized score from external data source,normalized,660,0.2
63,AMT_GOODS_PRICE,For consumer loans it is the price of the goods for which the loan is given,NaN,278,0.1
64,AMT_ANNUITY,Loan annuity,NaN,12,0.0
65,CNT_FAM_MEMBERS,How many family members does client have,NaN,2,0.0


In [180]:
#获取较小的缺失值列信息

t_small=t.loc[t["% of Total Values"]<1]
t_small


,Row,Description,Special,Missing Values,% of Total Values
57,NAME_TYPE_SUITE,Who was accompanying client when he was applying for the loan,NaN,1292,0.4
58,OBS_30_CNT_SOCIAL_CIRCLE,How many observation of client's social surroundings with observable 30 DPD (days past due) default,NaN,1021,0.3
59,DEF_30_CNT_SOCIAL_CIRCLE,How many observation of client's social surroundings defaulted on 30 DPD (days past due),NaN,1021,0.3
60,OBS_60_CNT_SOCIAL_CIRCLE,How many observation of client's social surroundings with observable 60 DPD (days past due) default,NaN,1021,0.3
61,DEF_60_CNT_SOCIAL_CIRCLE,How many observation of client's social surroundings defaulted on 60 (days past due) DPD,NaN,1021,0.3
62,EXT_SOURCE_2,Normalized score from external data source,normalized,660,0.2
63,AMT_GOODS_PRICE,For consumer loans it is the price of the goods for which the loan is given,NaN,278,0.1
64,AMT_ANNUITY,Loan annuity,NaN,12,0.0
65,CNT_FAM_MEMBERS,How many family members does client have,NaN,2,0.0
66,DAYS_LAST_PHONE_CHANGE,How many days before application did client change phone,NaN,1,0.0


In [181]:
#获取较小的缺失值列信息

t_large=t.loc[t["% of Total Values"]>1]
t_large

,Row,Description,Special,Missing Values,% of Total Values
0,COMMONAREA_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9
1,COMMONAREA_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9
2,COMMONAREA_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9
3,NONLIVINGAPARTMENTS_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,213514,69.4
4,NONLIVINGAPARTMENTS_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,213514,69.4
5,NONLIVINGAPARTMENTS_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,213514,69.4
6,FONDKAPREMONT_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,210295,68.4
7,LIVINGAPARTMENTS_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,210199,68.4
8,LIVINGAPARTMENTS_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,210199,68.4
9,LIVINGAPARTMENTS_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,210199,68.4


可以发现，残缺值数量差距很大，对于小于百分之一的残缺值，我们采取删去对应行的措施。


In [182]:
#删去部分行

app_tr.dropna(subset=t_small["Row"],
          axis=0, # axis=0表示删除行；
          how='any', # how=any表示若列name、age中，任意一个出现空值，就删掉该行
          inplace=True # inplace=True表示在原df上进行修改；
          )
app_tr = app_tr.reset_index(drop=True)
app_tr

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304526,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
304527,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
304528,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
304529,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


对于残缺值比较大的行，我们逐一进行研究处理。首先，对每个特征融入统计数据。

In [183]:

t_large_describe=t_large.merge(app_tr.describe().T,how="left",left_on="Row",right_index=True)
pd.set_option('max_colwidth',400)
t_large_describe


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,COMMONAREA_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9,91661.0,0.044544,0.076043,0.000000,0.007900,0.020800,0.051300,1.000000
1,COMMONAREA_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9,91661.0,0.044564,0.075932,0.000000,0.007800,0.021100,0.051400,1.000000
2,COMMONAREA_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,214865,69.9,91661.0,0.042510,0.074343,0.000000,0.007200,0.019000,0.048900,1.000000
3,NONLIVINGAPARTMENTS_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,213514,69.4,92987.0,0.008061,0.046265,0.000000,0.000000,0.000000,0.003900,1.000000
4,NONLIVINGAPARTMENTS_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,213514,69.4,92987.0,0.008795,0.047732,0.000000,0.000000,0.000000,0.003900,1.000000
5,NONLIVINGAPARTMENTS_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,213514,69.4,92987.0,0.008637,0.047412,0.000000,0.000000,0.000000,0.003900,1.000000
6,FONDKAPREMONT_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,210295,68.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LIVINGAPARTMENTS_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,210199,68.4,96272.0,0.105537,0.097673,0.000000,0.054200,0.076200,0.131300,1.000000
8,LIVINGAPARTMENTS_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,210199,68.4,96272.0,0.100662,0.092368,0.000000,0.050400,0.075600,0.121000,1.000000
9,LIVINGAPARTMENTS_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,210199,68.4,96272.0,0.101845,0.093431,0.000000,0.051300,0.076100,0.123100,

可以发现还存在一些存在大量残缺值的分类数据。对于这些数据，在转化为数值数据时，多转化一个类别。
另外，这里面有着大量缺失的房产数据，可以简化这些特征为是否有房产，注意可以考虑到只要是这些属性有一个就说明有房产。



In [184]:
#填补分类数据缺失值

temp=t_large_describe.drop(columns=["Special"]).isnull().T.any()
temp.loc[temp==True].index
rows=t_large_describe.loc[temp.loc[temp==True].index].Row

for col in rows: 
    app_tr[str(col)] = app_tr[str(col)].fillna(value="MyNull")
    
t=missing_values_table(app_tr,"application_{train|test}.csv")
app_tr[str(rows.iloc[0])]

Total 52 columns missing values


0         reg oper account
1         reg oper account
2                   MyNull
3                   MyNull
4                   MyNull
                ...       
304526    reg oper account
304527    reg oper account
304528    reg oper account
304529              MyNull
304530              MyNull
Name: FONDKAPREMONT_MODE, Length: 304531, dtype: object

In [185]:
#重新得到缺失值矩阵

t_describe=t.merge(app_tr.describe().T,how="left",left_on="Row",right_index=True)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304526,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
304527,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
304528,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
304529,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [186]:
t_house=t_describe.loc[t_describe["Description"]=="Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor"]
temp=app_tr.loc[:,t_house["Row"].tolist()].isnull().sum(axis=1)
app_tr=app_tr.drop(columns=t_house["Row"].tolist())
app_tr["MY_HOUSE_OWN"]=1
temp_app_list=app_tr["MY_HOUSE_OWN"].copy()
for index in temp.loc[temp==0].index:
    temp_app_list[index]=0
app_tr["MY_HOUSE_OWN"]=temp_app_list
app_tr["MY_HOUSE_OWN"]

0         0
1         0
2         1
3         1
4         1
         ..
304526    0
304527    0
304528    1
304529    1
304530    1
Name: MY_HOUSE_OWN, Length: 304531, dtype: int64

In [187]:
app_tr

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,MY_HOUSE_OWN
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304526,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
304527,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
304528,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0,1
304529,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [189]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
t_describe=t.merge(app_tr.describe().T,how="left",left_on="Row",right_index=True)
t_describe

Total 9 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,OWN_CAR_AGE,Age of client's car,NaN,200912,66.0,103619.0,12.070682,11.935821,0.000000,5.000000,9.000000,15.000000,91.000000
1,EXT_SOURCE_1,Normalized score from external data source,normalized,171652,56.4,132879.0,0.501986,0.211049,0.014568,0.333967,0.505819,0.674901,0.962693
2,EXT_SOURCE_3,Normalized score from external data source,normalized,60251,19.8,244280.0,0.510764,0.194843,0.000527,0.370650,0.535276,0.669057,0.896010
3,AMT_REQ_CREDIT_BUREAU_YEAR,Number of enquiries to Credit Bureau about the client one day year (excluding last 3 months before application),NaN,41108,13.5,263423.0,1.905904,1.869645,0.000000,0.000000,1.000000,3.000000,25.000000
4,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the client 3 month before application (excluding one month before application),NaN,41108,13.5,263423.0,0.266127,0.795735,0.000000,0.000000,0.000000,0.000000,261.000000
5,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the client one month before application (excluding one week before application),NaN,41108,13.5,263423.0,0.267782,0.915330,0.000000,0.000000,0.000000,0.000000,27.000000
6,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the client one week before application (excluding one day before application),NaN,41108,13.5,263423.0,0.034484,0.204615,0.000000,0.000000,0.000000,0.000000,8.000000
7,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the client one day before application (excluding one hour before application),NaN,41108,13.5,263423.0,0.006981,0.110358,0.000000,0.000000,0.000000,0.000000,9.000000
8,AMT_REQ_CREDIT_BUREAU_HOUR,Number of enquiries to Credit Bureau about the client one hour before application,NaN,41108,13.5,263423.0,0.006385,0.083786,0.000000,0.000000,0.000000,0.000000,4.000000
